In [ ]:
#WEB SCRAPPING y ALMACENAMIENTO SQL.

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import psycopg2

# Configuración de la base de datos
conn = psycopg2.connect(
    dbname="tu_base_de_datos",
    user="tu_usuario",
    password="tu_contraseña",
    host="tu_host",
    port="tu_puerto"
)
cursor = conn.cursor()

# Crear la tabla en la base de datos (ajusta según tu esquema)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS tablas_web (
        id SERIAL PRIMARY KEY,
        pagina TEXT,
        datos JSONB
    )
""")

# Instanciar el WebDriver de Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())

# URL de la página web
url = "https://www.example.com"
driver.get(url)

# Encontrar todas las tablas en la página
tables = driver.find_elements(By.TAG_NAME, "table")

for index, table in enumerate(tables):
    # Crear una lista para almacenar los datos de cada fila
    data = []
    for row in table.find_elements(By.TAG_NAME, "tr"):
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        data.append(row_data)

    # Crear un DataFrame de pandas
    df = pd.DataFrame(data)

    # Convertir el DataFrame a un diccionario para almacenarlo como JSON
    data_json = df.to_json(orient='records')

    # Insertar los datos en la base de datos
    sql = """
        INSERT INTO tablas_web (pagina, datos)
        VALUES (%s, %s)
    """
    val = (url, data_json)
    cursor.execute(sql, val)

# Confirmar los cambios
conn.commit()

# Cerrar la conexión a la base de datos y el navegador
cursor.close()
conn.close()
driver.quit()